In [1]:
import json
import nltk
import re
import numpy as np
import tensorflow as tf
from prepro import convert_idx

In [2]:
with open('/home/soham/data/squad/sample1k-HCVerifyAll.json', 'r') as f:
    foo = json.load(f)

In [3]:
meh = []

In [4]:
foo['version']

'1.1'

In [5]:
# i = 0; j = 0
# original_paragraphs = {}
# adversarial_paragraphs = {}
# titles = set()

# def strip_id(id_):
#     return id_.split('-')[0]

# for datum in foo['data']:
#     titles.add(datum['title'])
    
#     for paragraph in datum['paragraphs']:
#         context = paragraph['context']
        
#         for qa in paragraph['qas']:
#             answers = qa['answers']
#             question = qa['question']
#             id_ = qa['id']
            
#             if strip_id(id_) == id_:
#                 assert id_ not in original_paragraphs
#                 original_paragraphs[id_] = (id_, datum['title'], context, question, answers)
#                 i += 1
#             else:
#                 if strip_id(id_) not in adversarial_paragraphs:
#                     adversarial_paragraphs[strip_id(id_)] = []
                    
#                 adversarial_paragraphs[strip_id(id_)] += [(context, question, answers)]
#                 j += 1

# print(i, j)

In [6]:
# np.random.seed(1337)
# train_titles = set(np.random.choice(list(titles), size=int(len(titles)*0.7), replace=False))
# test_titles = titles - train_titles

In [7]:
# print(', '.join(list(train_titles)))
# print()
# print(', '.join(list(test_titles)))

3560 total:  

* 2560 mutated
* 1000 original

In [8]:
# train_pairs = []
# test_pairs = []

# for key, (id_, title, orig_para, orig_ques, orig_ans) in original_paragraphs.items():
#      #id_, title, orig_para, orig_ques, orig_answer = id_title_para_ques_ans
    
#     if title in train_titles:
#         train_pairs += [(id_, orig_para, orig_ques, None, 'orig', title)]
#     else:
#         assert title in test_titles
#         test_pairs += [(id_, orig_para, orig_ques, orig_ans, None, 'orig', title)]
    
#     if key in adversarial_paragraphs:
#         for j, (para, ques, ans) in enumerate(adversarial_paragraphs[key]):
#             injection = para[len(orig_para)+1:]
            
#             orig_sents = nltk.sent_tokenize(orig_para)
            
#             i = np.random.randint(0, len(orig_sents) + 1)
#             adv_para = ' '.join(orig_sents[:i] + [injection] + orig_sents[i:])
            
#             if i == 0:
#                 start = 0
#             else:
#                 start = len(' '.join(orig_sents[:i])) + 1
            
#             end = start + len(injection)
#             die_counter = 0
#             for j in range(len(ans)):
#                 if start <= ans[j]['answer_start']:
#                     new_start = ans[j]['answer_start'] + len(injection) - 1
#                     pass #ans[j]['answer_start'] += len(injection)
#                 else:
#                     new_start = ans[j]['answer_start']
                
#                 die_counter = 0
                
#                 while not adv_para[new_start:].startswith(ans[j]['text']):
#                     new_start += 1
#                     die_counter += 1
                    
#                     if die_counter == 9999:
#                         break
                
#                 if die_counter == 9999:
#                     break
                
#                 assert adv_para[new_start:].startswith(ans[j]['text']), (adv_para[new_start:], ans[j]['text'])
#                 ans[j]['answer_start'] = new_start
            
#             if die_counter == 9999:
#                 print('YOU DIED')
#                 print(adv_para)
#                 print(orig_para)
#                 print()
#                 continue
            
#             if title in train_titles:
#                 if True: #np.random.randint(3, size=1)[0] == 1:
#                     train_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, (start, end), 'adv', title)]
#             else:
#                 assert title in test_titles
#                 if True: #np.random.randint(3, size=1)[0] == 1:
#                     test_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, ans, (start, end), 'adv', title)]

In [9]:
train_pairs = np.load('final_train_pairs.npy').tolist()
test_pairs = np.load('final_test_pairs.npy').tolist()

In [10]:
adv_count = lambda x: sum(map(lambda y: 1 if y[-2] == 'adv' else 0, x))

In [11]:
print('Train:', '%d,' % len(train_pairs), adv_count(train_pairs), 'of which are adversarial')
print('Test:', '%d,' % len(test_pairs), adv_count(test_pairs), 'of which are adversarial')

Train: 2178, 1570 of which are adversarial
Test: 1381, 989 of which are adversarial


In [12]:
import spacy
nlp = spacy.blank("en")

In [13]:
def word_tokenize(sent):
    doc = nlp(sent)
    return [token.text for token in doc]

In [14]:
def _get_word(word):
    for each in (word, word.lower(), word.capitalize(), word.upper()):
        if each in word2idx_dict:
            return word2idx_dict[each]
    return 1

def _get_char(char):
    if char in char2idx_dict:
        return char2idx_dict[char]
    return 1

In [15]:
with open('data/word2idx.json', 'r') as fh:
    word2idx_dict = json.load(fh)

with open('data/char2idx.json', 'r') as fh:
    char2idx_dict = json.load(fh)

In [16]:
para_limit = 1000
ques_limit = 100
char_limit = 16

In [17]:
import random
random.shuffle(train_pairs)
random.shuffle(test_pairs)

In [18]:
%rm data/badptr_t*.tf

In [19]:
writer = tf.python_io.TFRecordWriter('data/badptr_train.tf')

for id_, (id_but_ignore, context, ques, ans, start_end, tag, title) in enumerate(train_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    
    if start_end is None:
        y1[0] = 1
        y2[0] = 1
    else:
        spans = convert_idx(context, context_tokens)
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        y1[answer_span[0]] = 1
        y2[answer_span[-1]] = 1
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()

In [20]:
from prepro import save

In [23]:
writer = tf.python_io.TFRecordWriter('data/badptr_test.tf')
classic_meta_info = {}
meta_info = {}

for id_, (id_but_ignore, context, ques, ans, start_end, tag, title) in enumerate(test_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    spans = convert_idx(context, context_tokens)
    
    if start_end is None:
        y1[0] = 1
        y2[0] = 1
    else:
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        y1[answer_span[0]] = 1
        y2[answer_span[-1]] = 1
    
    if start_end is None:
        new_answer_ = context[spans[0][0]:spans[0][1]]
    else:
        new_answer_ = context[spans[answer_span[0]][0]:spans[answer_span[-1]][1]]
    
    meta_info[id_] = {'answers': [new_answer_], 'spans': spans, 'context': context, 'uuid': id_but_ignore}
    classic_meta_info[id_] = {'answers': ans, 'spans': spans, 'context': context, 'ques': ques, 'tag': tag, 'title': title, 'id': id_}
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()
save('data/badptr_test_meta.json', meta_info, message='meta_info')
np.save('data/badptr_test_meta.npy', classic_meta_info)

Saving meta_info...


In [24]:
len(set([a for a in map(lambda x: x['uuid'], meta_info.values())]))

1381